In [1]:
from utz import *

In [1]:
import duckdb
duckdb.sql("INSTALL httpfs;")
duckdb.sql("INSTALL s3fs;")

HTTPException: HTTP Error: Failed to download extension "s3fs" at URL "http://extensions.duckdb.org/v0.9.2/osx_arm64/s3fs.duckdb_extension.gz"

Candidate extensions: "s3", "fts", "substrait", "aws"

In [3]:
duckdb.sql("LOAD httpfs;")
duckdb.sql("LOAD s3fs;")

IOException: IO Error: Extension "/Users/ryan/.duckdb/extensions/v0.9.2/osx_arm64/s3fs.duckdb_extension" not found.

Candidate extensions: "s3", "fts", "substrait", "aws"

In [5]:
con = duckdb.connect('s3://nj-crashes/njdot/2021.duckdb', read_only=True)
dql = con.sql

CatalogException: Catalog Error: Cannot open database "/Users/ryan/c/neighbor-ryan/nj-crashes/njdot/s3://nj-crashes/njdot/2021.duckdb" in read-only mode: database does not exist

In [10]:
con = duckdb.connect('s3://nj-crashes/njdot/data/2021/NewJersey2021Accidents.pqt', read_only=True)
dql = con.sql

CatalogException: Catalog Error: Cannot open database "/Users/ryan/c/neighbor-ryan/nj-crashes/njdot/s3://nj-crashes/njdot/data/2021/NewJersey2021Accidents.pqt" in read-only mode: database does not exist

In [9]:
dql("IMPORT DATABASE 's3://nj-crashes/crashes.ddb'")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [10]:
dql("SHOW TABLES")

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ crashes │
└─────────┘

In [11]:
dql("DESCRIBE crashes")

┌───────────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│        column_name        │ column_type │  null   │   key   │ default │ extra │
│          varchar          │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ year                      │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ cc                        │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ mc                        │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ case                      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ cn                        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ mn                        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ pdc                       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ pdn                       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ station       

In [12]:
dql("SELECT * from duckdb_indexes()").df()

,database_name,database_oid,schema_name,schema_oid,index_name,index_oid,table_name,table_oid,is_unique,is_primary,expressions,sql
0,memory,992,main,994,crashes_id_idx,1393,crashes,1389,True,False,None,\nCREATE UNIQUE INDEX crashes_id_idx ON crashe...
1,memory,992,main,994,crashes_dt_icn_severity_idx,1391,crashes,1389,False,False,None,\n\n\nCREATE INDEX crashes_dt_icn_severity_idx...


In [13]:
%%time
dql("""
SELECT date_part('year', dt) as year, icn as county, severity, count(*) as num
FROM crashes
WHERE icn IS NOT NULL
GROUP BY (date_part('year', dt), icn, severity)
ORDER BY (date_part('year', dt), icn, severity)
""")

CPU times: user 887 µs, sys: 945 µs, total: 1.83 ms
Wall time: 1.78 ms


┌───────┬────────────┬──────────┬───────┐
│ year  │   county   │ severity │  num  │
│ int64 │  varchar   │ varchar  │ int64 │
├───────┼────────────┼──────────┼───────┤
│  2001 │ Atlantic   │ f        │    49 │
│  2001 │ Atlantic   │ i        │  1689 │
│  2001 │ Atlantic   │ p        │  2860 │
│  2001 │ Bergen     │ f        │    16 │
│  2001 │ Bergen     │ i        │  3181 │
│  2001 │ Bergen     │ p        │  7189 │
│  2001 │ Burlington │ f        │    54 │
│  2001 │ Burlington │ i        │  1829 │
│  2001 │ Burlington │ p        │  4050 │
│  2001 │ Camden     │ f        │    33 │
│    ·  │   ·        │ ·        │     · │
│    ·  │   ·        │ ·        │     · │
│    ·  │   ·        │ ·        │     · │
│  2021 │ Somerset   │ p        │  5506 │
│  2021 │ Sussex     │ f        │     9 │
│  2021 │ Sussex     │ i        │   423 │
│  2021 │ Sussex     │ p        │  1591 │
│  2021 │ Union      │ f        │    28 │
│  2021 │ Union      │ i        │  3698 │
│  2021 │ Union      │ p        │ 

In [22]:
%%time
c = dql("""
SELECT sri, mp, ilat, ilon, sum("Total Killed") as tk, sum("Total Injured") as ti
FROM crashes
WHERE icn='Hudson' AND ("Total Killed" > 0 OR "Total Injured" > 0)
AND date_part('year', dt) >= 2017
GROUP BY sri, mp, ilat, ilon
""").df()
c

CPU times: user 116 ms, sys: 122 ms, total: 238 ms
Wall time: 112 ms


,sri,mp,ilat,ilon,tk,ti
0,09011544__,1.16,40.667220,-74.115474,0.0,5.0
1,09011543__,0.77,40.664319,-74.121225,0.0,12.0
2,09011544__,0.10,40.654347,-74.126355,0.0,2.0
3,09011166__,2.12,40.652183,-74.137501,0.0,2.0
4,00000501__,26.74,40.688826,-74.106185,0.0,3.0
...,...,...,...,...,...,...
4741,09111118__,0.57,40.770592,-74.015398,0.0,2.0
4742,09121012__,0.38,40.789431,-74.010708,0.0,1.0
4743,09011073__,0.00,40.656370,-74.121343,0.0,1.0
4744,09061223__,0.60,40.707208,-74.084650,0.0,1.0


In [25]:
%%time
hc = dql("""
SELECT id, dt, sri, mp, ilat, ilon
FROM crashes
WHERE icn='Hudson'
AND date_part('year', dt) >= 2017
""").df().set_index('id')
hc

CPU times: user 107 ms, sys: 5.92 ms, total: 112 ms
Wall time: 33.9 ms


,dt,sri,mp,ilat,ilon
id,,,,,
4821060,2017-02-09 09:44:00,00000003__,8.49,40.798442,-74.068342
4821092,2017-03-28 17:33:00,00000003__,8.49,40.798442,-74.068342
4821173,2017-08-26 08:41:00,00000003__,8.49,40.798442,-74.068342
4821234,2017-10-11 15:51:00,00000003__,8.49,40.798442,-74.068342
4821266,2017-10-30 15:37:00,00000003__,8.49,40.798442,-74.068342
...,...,...,...,...,...
5958480,2021-04-01 13:19:00,09121045__,0.47,40.787305,-74.014560
5958481,2021-06-25 21:42:00,00000501__,35.35,40.788598,-74.023481
5958482,2021-09-24 12:46:00,00000501__,35.99,40.795783,-74.016394


In [27]:
%%time
dql("""
SELECT severity, count(*)
FROM crashes
WHERE cn='Hudson'
AND icn IS NULL
AND date_part('year', dt) >= 2017
GROUP BY severity
""")

CPU times: user 384 µs, sys: 34 µs, total: 418 µs
Wall time: 425 µs


┌──────────┬──────────────┐
│ severity │ count_star() │
│ varchar  │    int64     │
├──────────┼──────────────┤
│ f        │           28 │
│ i        │         3954 │
│ p        │        24976 │
└──────────┴──────────────┘